# Jarque Bera test for normality of returns in equities
***

In [1]:
import pandas as pd 
import numpy as np
import scipy.stats as stats
import matplotlib.pyplot 
import yfinance as yf
from returns import log_returns
from datetime import date
from dateutil.relativedelta  import relativedelta


### The Jarque-Bera Test


- The Jarque-Bera (JB) test is a statistical test that can be used to test whether a given sample was drawnfrom a normal distribution.
- The null hypothesis is that the data set has the same skewness (0) and kurtosis (3) as a normaldistribution.
 - The test statistic is:

\begin{equation*}
    JB = \frac{n}{6}( S^2 + \frac{1}{4} (K-3)^2    )
\end{equation*}

where $S$ is the sample skewness, $K$ is the sample kurtosis, and $n$ is the number of observations.
- It is implemented in scipy.stats.jarque_bera()

### Implementation

The JB test statistic is computed in the same method as in the previous exercise, which is done by computing the skewness and kurtosis of the returns and then using the JB formula compute the test statistic. The other values which has to be computed is the p-value, where the p-value is the probability of obtaining results at the extreme. The p-value is calculated by obtaining the area underneath the distribution for the extreme observation. This is computed by counting the number of JB bootstrap test statistics that are less than the JB empirical test statistic. Once we obtain the number of terms that less we scale the value by dividing it by the number of samples and subtract it from the total area of the distribution which is one. Lastly we have to observe the distributution by plotting the boxplot for each sampling frequency then comparing them to observe if stock returns exhibit aggregational Gaussianity. Aggregational Gaussianity is when the time step of the returns increases, this should lead to a distribution which looks more like normal distribution.

Once the test statistics and p-values are computed the test statistics have to be plotted as a boxplot to allow comparison between the different sampling frequencies. As the values have to be computed for different sampling frequencies a function will be used which will compute the test statistic and p-value for each stock. Then using a loop, the function will be used to compute the values for all the stocks and finally be represented as dataframe.

First step is to define the function. The function arguments are the data source of the returns, the stock ticker for the current stock in the loop, the sampling frequency (eg. 'M' corresponding to monthly) and final argument being the number of bootstrap replications. Next we compute the size of the returns by using the count function to count how many prices we have in the stock, as this method excludes any Nan values.

The empirical value is computed by computing the skewness and kurtosis along with a omitting Nan parameters so that Nan values are dropped. The skewness and kurtosis is substituted in the JB formula to obtain the empirical test statistic.

To compute the p-value requires computing the bootstrap data distribution. First, using the number of bootstrap replications and the size of the data, we draw sets of data from a normal distribution. Then as before the skewness and kurtosis is computed, then substituted in to the JB formula to give us a distribution of JB test statistics.

To compute the p-values, we use a for loop to compare the current JB test statistic to the empirical test statistic. Then using a if statement we add one to the counter if the current JB statistic in the loop of the bootstrap distribution is less than the empirical test statistic. This value is then scaled by dividing it by the size of the samples, then subtracted from the total area which is 1, to give us the p-value.

A result dataframe is created so that the function returns a dataframe containing the test statistic and p-values for the stock. This function is called in a for loop so it computes the values for all stocks i the index.

In [86]:
# Define function for Jarque-Bera test
def JB(data_source, T, bootstrap):

    data  = data_source
    # Compute size of data 
    size_data  = int(data.count())
    
    # Empirical value
    s_emp = stats.skew(data, nan_policy='omit') # Obtain skewnness of returns with parameter nan-policy set to omit to exclude any nan in data
    k_emp = stats.kurtosis(data, fisher = False, nan_policy='omit') # Obtain kurtosis of returns with parameter nan-policy set to omit to exclude any nan in data
    JB_emp = (size_data/6) *  (s_emp**2 + (((k_emp-3)**2)/4)) # Compute test statistic for empirical data
    
    # Compute the bootstrap data
    bootstrap_data = np.random.normal(size=(size_data, bootstrap))
    s_boot = stats.skew(bootstrap_data, nan_policy='omit') # Obtain skewnness of theoretical distribution with parameter nan-policy set to omit to exclude any nan in data
    k_boot = stats.kurtosis(bootstrap_data, fisher = False, nan_policy='omit') # Obtain kurtosis of theoretical distribution with parameter nan-policy set to omit to exclude any nan in data
    JB_boot = (size_data/6) *  (s_boot**2 + (((k_boot-3)**2)/4)) # Compute test statistics for theoretical distribution
    
    # Compute p-values
    counter = 0.0
    for i in range(len(JB_boot)):
        if JB_boot[i] < JB_emp:
            counter = counter + 1.0
    
    p_value = 1 - (counter/len(JB_boot)) 

    result = pd.DataFrame({"Stock values" : [JB_emp,p_value]}, index = ["%s JB test"%T,"%s p-value"%T ])

    return result

### Loading data


In [99]:
# Use Goldman Sachs ticker
ticker = 'GS'
security_object = yf.Ticker(ticker)

# Retrieve historical prices for selected ticker
security_prices = security_object.history(start=(date.today() - relativedelta(years=5)))
# Only use close prices
df_prices = security_prices['Close']

df_prices.head(3)

Date
2017-12-04    226.496170
2017-12-05    224.399750
2017-12-06    222.249069
Name: Close, dtype: float64

### JB for daily returns

In [100]:
returns_GS_daily = log_returns(df_prices, 'D')
jb_daily = JB(returns_GS_daily, 'Daily', 5000)
jb_daily

,Stock values
Daily JB test,[4589.941840945824]
Daily p-value,0.0


### JB for monthly returns

In [101]:
returns_GS_monthly = log_returns(df_prices, 'M')
jb_monthly = JB(returns_GS_monthly, 'Monthly', 5000)
jb_monthly

,Stock values
Monthly JB test,[0.6240025928146146]
Monthly p-value,0.6968


### JB for annual returns

In [102]:
returns_GS_annual = log_returns(df_prices, 'A')
jb_annual = JB(returns_GS_annual, 'Annual', 5000)
jb_annual

,Stock values
Annual JB test,[0.6390608573641522]
Annual p-value,0.3124


### JB results

In [103]:
JB_statistics = pd.concat([jb_daily, jb_monthly, jb_annual])

JB_statistics_transpose = JB_statistics.transpose()

JB_statistics_transpose

,Daily JB test,Daily p-value,Monthly JB test,Monthly p-value,Annual JB test,Annual p-value
Stock values,[4589.941840945824],0.0,[0.6240025928146146],0.6968,[0.6390608573641522],0.3124
